# Row- or Column-major order
It is the way computer store the multidimensional vectors.  In general, there are two ways to do so: (1) Row major order; (2) Column major order.  See an example below:
https://en.wikipedia.org/wiki/Row-major_order.
Julia uses Column-major order.  

In [3]:
function proj(u,v)
    return (dot(u,u)/dot(u,u))*u
end
A = [1 2; 10 50]
A[:,1]

2-element Array{Int64,1}:
  1
 10

> ```A[:,1]``` is a column vector and hence, Julia treats it as a one dimensional vector as the type is Array{Int64,1}.  

In [4]:
A[1,:]

1x2 Array{Int64,2}:
 1  2

> Since A[1,:] is a row vector, Julia treats it as a "two" dimensional array as the type is Array{Int64,2}.

In [5]:
dot(A[1,:], A[2,:]) # will give an error because of the wrong type

LoadError: LoadError: MethodError: `dot` has no method matching dot(::Array{Int64,2}, ::Array{Int64,2})

You might have used a 2d row vector where a 1d column vector was required.
Note the difference between 1d column vector [1,2,3] and 2d row vector [1 2 3].
You can convert to a column vector with the vec() function.
while loading In[5], in expression starting on line 1

In [6]:
dot(A[:,1], A[:,2]) # will be OK!

502

# Gram-Schmidt Process
Given a linear independent set $\{ x_1, x_2, \dots, x_n\}$, construct an orthonormal set $\{ u_1, u_2, \dots, u_n \}$.

* $v_1 = x_1$
* $v_2 = x_2 - proj_{v_1}(x_2)$
* $v_3 = x_3 - proj_{v_1}(x_3) - proj_{v_2}(x_3)$
* $\vdots$
* $v_n = x_n - \sum_{i=1}^{n-1} proj_{v_i}(x_n)$

Lastly, normalize $v_1, \dots, v_n$:
* $u_i = \frac{v_i}{\|v_i\|_2}$



In [7]:
function GramSchmidt(X)
    #Suppose X is a matrix of size n by m. n is dim of vectors and m is the number of vectors
    #Initialize...
    OX = zeros(size(X))
    nrow, ncol = size(X)
    
    #First step...
    OX[:,1] = X[:,1]
    
    #Rest of Steps
    for i = 2:ncol
        tempOX = zeros(nrow)
        for j = 1:i-1
            tempOX = tempOX + proj(OX[:,j], X[:,i])
        end
        OX[:,i] = X[:,i] - tempOX
    end
    
    #Normalize
    for i = 1:ncol
        OX[:,i] = OX[:,i] / norm(OX[:,i])
    end
    
    #Return Orthogonal matrix
    return OX
end

GramSchmidt (generic function with 1 method)

In [8]:
X = [1 1 1; 0 1 1; 0 0 1]
GramSchmidt(X)

3x3 Array{Float64,2}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

# Newton's Method
Newton's method is designed for solving the root of a nonlinear function $F(x)$.  It is an iterative method and can be written as for an inital guess $x_0$,
$$x_{i+1} = x_i - [DF(x_i)]^{-1} F(x_i)$$,
where $DF$ is the Jacobian matrix of the function $F$.

In [61]:
function newton(F, DF, x, TOL)
    xp = zeros(size(x))
    while norm(F(x)) > TOL && norm(x-xp) > TOL
        xp[:] = x[:]
        x[:] = x[:] - (DF(x)\F(x)) # inv is inefficient
        #println(norm(x-xp, Inf))
    end
    return x
end

newton (generic function with 1 method)

$$
\begin{cases}
3x_1 - \cos(x_2 x_3) - 0.5 &= 0 \\
x_1^2 - 81(x_2 + 0.1)^2 + \sin(x_3) + 1.06 &= 0 \\
e^{-x_1 x_2} + 20x_3 + (10\pi -3 )/3 & = 0
\end{cases},
$$
with the initial guess $x^0 = [0.1 0.1 -0.1]^T$.  

The Jacobian matrix $J(x)$ for this system is 
$$
J(x_1, x_2, x_3) =
\begin{bmatrix}
3 & x_3 \sin(x_2 x_3) & x_2 \sin(x_2 x_3) \\
2x_1 & -162(x_2 + 0.1) & \cos(x_3) \\
-x_2 e^{-x_1 x_2} & -x_1 e^{-x_1 x_2} & 20
\end{bmatrix}
$$

In [16]:
function F(x)
    return [ 3*x[1] - cos(x[2]*x[3]) - 1/2; x[1]*x[1] - 81*(x[2]+0.1)^2 + sin(x[3])+1.06; 
        exp(-x[1]*x[2]) + 20*x[3] + (10*pi-3)/3 ]
end

function DF(x)
    mat = [ 
        3 x[3]*sin(x[2]*x[3]) x[2]*sin(x[2]*x[3]); 
        2*x[1] -162*(x[2]+0.1) cos(x[3]);
        -x[2]*exp(-x[1]*x[2]) -x[1]*exp(-x[1]*x[2]) 20
    ];
    return mat
end

DF (generic function with 1 method)

In [11]:
newton(F, DF, [0.1;0.1;-0.1], 1e-12)

0.42152047193583064
0.017878257167124205
0.0015761465869723393
1.244400753583079e-5
7.757857127143586e-10


Good thing about Newton's method:
* Quadratic convergence.

Things to watch:
* Good initial guess.
* Jacobian matrix is expensive.

# 7c.

$$
x^3_1 + x^2_1 x_2 - x_1 x_3 + 6 = 0 \\
e^{x_1} + e^{x_2} - x_3 = 0 \\
x^2_2 - 2x_1 x_3 = 4
$$

$x^{(0)} = (-1, -2, 1)^t$

In [65]:
function F(x)
    return [
        x[1]^3 + (x[1]^2)*x[2] - x[1]*x[3] + 6;
        e^(x[1]) + e^(x[2]) - x[3];
        x[2]^2 - 2*x[1]*x[3] - 4;
    ];
end

function DF(x)
    return [
        3*(x[1]^2)+2*x[1]*x[2]-x[3] x[1]^2 -x[1];
        e^(x[1]) e^(x[2]) -1;
        -2*x[3] 2*x[2] -2x[1]
        ];
end

x0 = [-1.0; -2; 1]
TOL = 1e-7

1.0e-7

In [68]:
newton(F, DF, x0, TOL)

3-element Array{Float64,1}:
 -1.45604 
 -1.66423 
  0.422493

In [76]:
function F(x)
    return [
        4*x[1] - x[2] + x[3] - x[1]*x[4];
        -x[1] + 3*x[2] - 2*x[3] - x[2]*x[4];
        x[1] - 2*x[2] + 3*x[3] - x[3]*x[4];
        x[1]*2 + x[2]*2 + x[3]*2 - 1
    ];
end
function DF(x)
    return [
        4-x[4] -1 1 -x[1];
        -1 3+x[4] -2 -x[2];
        1 -2 3-x[4] -x[4];
        2*x[1] 2*x[2] 2*x[3] 0;        
    ];

end
TOL = 1e-6
x0 = [1.0;1;1;1]

4-element Array{Float64,1}:
 1.0
 1.0
 1.0
 1.0

In [77]:
newton(F, DF, x0, TOL)

4-element Array{Float64,1}:
 NaN          
  -7.00481e154
  -9.87907e154
  14.9929     